##### _Group #36 Project Proposal_
## Heart Disease Dataset

In [2]:
# These packages need to be loaded

library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)
library(GGally)

options(repr.matrix.max.rows = 6)
set.seed(1)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

# _1) Introduction:_

* Heart disease is an increasingly prevalent and crippling disease accepting a multitude of different people across the globe. Given its severity, the ability to accurately diagnose the disease early is becoming increasingly important to ensure the longevity of these affected individuals. 

* Within our project we are looking to see if age and maximum heart rate achieved can accurately predict the diagnosis of heart disease. 

* The data set we will be using is a compiled data set about Heart Disease which we found on KAGGLE. It consists of different onset heart disease indicators with data from multiple different countries and spanning from 1988 onwards. The data was measured upon hospital admission for acute chest pain. This data set includes data on the age, sex, cheat pain type from 0-4, the resting blood pressure, serum cholestoral level in the blood, the fasting blood sugar, the resting electrocardiographic results with values 0-2, the maximum heart rate achieved, the presence of exercise induced angina, data about results of electrocartiogram tests including the ST depression value and the slope of the ST segment, # of major vessels from 0-3, and the diagnosis of the blood disorder thalassemia with 0 as normal, 1 as fixed defect and 2 as reversable defect. 

* Finally, the last column, target column, represents the presence of heart disease in the patient with the following two cases:
            1. 0 = no disease
            2. 1 = disease



------------------------------------------------------------------------------------------------------------------


# _2) Preliminary exploratory data analysis:_

##### Below we demonstrate that the dataset can be read from the web into R through a URL. We clean and wrangle our data into a tidy format by removing the repeated observations in the set. It was stated on the website where we retrieved the data from therefore we removed repeated observations using unique() function. This decreases the total rows from 1025 to 302. Otherwise, our data is already in a tidy format because in each column there is one variable and each row has one observation



In [3]:
url <- "https://raw.githubusercontent.com/sadia-kd/DSCI-Project/main/heart.csv"

heart_disease <- read_csv(url) %>%
    mutate(target = as_factor(target)) %>%
    unique()

heart_disease

Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  cp = col_double(),
  trestbps = col_double(),
  chol = col_double(),
  fbs = col_double(),
  restecg = col_double(),
  thalach = col_double(),
  exang = col_double(),
  oldpeak = col_double(),
  slope = col_double(),
  ca = col_double(),
  thal = col_double(),
  target = col_double()
)



age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
52,1,0,128,255,0,1,161,1,0.0,2,1,3,0
59,1,3,160,273,0,0,125,0,0.0,2,0,2,0
54,1,0,120,188,0,1,113,0,1.4,1,1,3,0


### Explore The Data

* Here, I summarized the data in a table (exploratory data analysis). We have a table that reports the number of observations in each class, in the training data


In [4]:
# Here we split the data into training and testing data so we can further use only
# training data for exploratory data analysis

heart_disease_split <- initial_split(heart_disease, prop = 0.75, strata = target)
heart_disease_train <- training(heart_disease_split)
heart_disease_test <- testing(heart_disease_split) 

In [5]:
# Summarize the DATA 

obs <- nrow(heart_disease_train)

data_count <- heart_disease_train %>%
    group_by(target) %>%
    summarize(count = n(), percentage = n()/obs * 100)

data_count

summary(heart_disease_train)

`summarise()` ungrouping output (override with `.groups` argument)



target,count,percentage
<fct>,<int>,<dbl>
0,104,45.81498
1,123,54.18502


      age             sex               cp            trestbps    
 Min.   :29.00   Min.   :0.0000   Min.   :0.0000   Min.   : 94.0  
 1st Qu.:47.00   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:120.0  
 Median :55.00   Median :1.0000   Median :1.0000   Median :130.0  
 Mean   :54.25   Mean   :0.6784   Mean   :0.9163   Mean   :130.9  
 3rd Qu.:61.00   3rd Qu.:1.0000   3rd Qu.:2.0000   3rd Qu.:140.0  
 Max.   :74.00   Max.   :1.0000   Max.   :3.0000   Max.   :200.0  
      chol            fbs            restecg          thalach     
 Min.   :126.0   Min.   :0.0000   Min.   :0.0000   Min.   : 71.0  
 1st Qu.:211.0   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:139.5  
 Median :241.0   Median :0.0000   Median :1.0000   Median :154.0  
 Mean   :249.0   Mean   :0.1498   Mean   :0.5463   Mean   :150.7  
 3rd Qu.:279.5   3rd Qu.:0.0000   3rd Qu.:1.0000   3rd Qu.:167.5  
 Max.   :564.0   Max.   :1.0000   Max.   :2.0000   Max.   :202.0  
     exang           oldpeak          slope            ca     

This illustrates that in this data set, about 46% do not have a heart disease (0) , and 54 % do have a heart disease (1)


-----------------------------------------------------------------------------------------------------------------


### Visualize the Data

* Using only training data, we visualize the data relevant to our analysis we plan to do. An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [6]:
# I Scale and Center the DATA here and use this to get more affective plots

scaled_heart_disease <- recipe(target  ~ ., data = heart_disease_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors()) %>%
    prep() %>%
    bake(heart_disease_train)

scaled_heart_disease

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
-0.2480398,0.6869773,-0.8902681,-0.3312515,-0.6896746,-0.4187956,0.8786896,0.7605765,-0.7219306,-0.01714278,0.9412078,1.3690368,1.141504,0
-0.1378538,0.6869773,-0.8902681,0.5123621,-0.8573136,2.3772808,-1.0578399,0.1883018,1.3790726,1.84012128,-2.2476604,-0.7236996,1.141504,0
1.7353079,0.6869773,-0.8902681,0.7935667,-1.3974835,-0.4187956,0.8786896,-1.1323320,1.3790726,1.39791555,-2.2476604,-0.7236996,1.141504,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
-0.24803981,0.6869773,-0.8902681,-0.1625288,0.1112670,-0.4187956,0.8786896,0.4524286,1.3790726,-0.9015542,0.9412078,0.3226686,1.141504,0
0.52326207,0.6869773,2.0245040,1.6371804,0.4465448,-0.4187956,-1.0578399,-1.1323320,-0.7219306,-0.9015542,0.9412078,-0.7236996,-0.498505,0
-0.02766785,0.6869773,-0.8902681,-0.6124561,-1.1367118,-0.4187956,0.8786896,-1.6605855,-0.7219306,0.3366218,-0.6532263,0.3226686,1.141504,0


In [7]:
# I plot a graph of cholesterol levels vs. maximum heart rate achieved

plot_1 <- scaled_heart_disease %>%
    ggplot(aes(x = thalach, y =chol, color = target)) +
    geom_point() +
    labs(x = "Maximum Heart Rate Achieved", y = "Cholesterol Levels", color = "Heart Diease or No") +
    ggtitle("Cholesterol Levels vs Maximum Heart Rate Achieved")

In [8]:
plot_2 <- scaled_heart_disease %>%
    ggplot(aes(x = age, y = trestbps, color = target)) +
    geom_point() +
    labs(x = "Age", y = "Resting Blood Presure (mm Hg)", color = "Heart Disease or NO")

In [9]:
plot_3 <- scaled_heart_disease %>%
    ggplot(aes(x = age, y = oldpeak, color = target)) +
    geom_point() +
    labs(x = "Age", y = "ST depression", color = "Heart Disease or NO")

In [ ]:
# Plot all predictors against each other
options(repr.plot.width = 16, repr.plot.height = 10) 

plots <- ggpairs(scaled_heart_disease, mapping = NULL, columns = 1:13) 

plots


------------------------------------------------------------------------------------------------------------------


# _3) METHODS:_

### Data analysis steps:

1. Formulate a precise question that we want to answer:
    - Can age and maximum heart rate achieved can accurately predict the diagnosis of heart disease?
2. Loading the data: 
    - start by loading the tidyverrse package
    - Read the dataset using read_csv with no additional arguments, and then inspect its contents
    - Remove duplicated data from the dataset
    - Use function glimpse(dataset) to preview the data frame
    - Use function pull to extract a single column (target) and pass that into levels function to see the categories in the target column. 
3. Split the dataset into training data and testing data
    - Set the seed as set.seed(1)
    - Use the initial_split function to create the training and testing sets
4. Exploring the data
    - Use the group_by, summarize and n function to find the number and percentage of having heart disease(integer value = 1) or not (integer value = 0).
    - Draw a scatter plot to visualize the relationship between age and maximum heart rate achieved variables.
    - Based on the visualization, decide if these are useful variable for prediction
    - Classification with K-nearest neighbors
5. Choose a K value
    - Choose some candidate values of K   
    - Split the traning data into two sets, traning set and validation set
    - For each K, tran the model using traning set only
    - Evaluate accuracy for each using validation set only
    - Apply corss-validation technique - splitting multiple ways and averaging
    - Pick the K that maximizes validation accuracy
6. Classification
    - Load library tidymodels
    - Pick out our two desired predictor variables and class label and store them as a new data set
    - Create a model specification for K-nearest neighbors classification by calling the nearest_neighbor function.
    - Pass the model specification and the dataset to the fit function
    - Now we can use the predict function to predict our testing data.
7. Result analysis
    - Use the bind_cols function to add the column of predictions to the original test data
    - Create a confusion matrix by using the conf_mat function. Calculate accuracy = (TP+TN) / total number of test result 
8. Conclusion
    - Review our model
    - Review our goal how accurate age and maximum heart rate achieved can predict if the patient have heart disease, based on the data in this dataset
    - Finalize report

### Reason to choose age and maximum heart rate achieved:

In [ ]:
heart_disease_var_check <- data_train %>%
    ggplot(aes(x = thalach, y=age, color = target)) +
    geom_point() +
    labs(x = "Maximum Heart Rate Achieved", y = "Age", color = "Heart Diease or No") +
    ggtitle("Maximum Heart Rate Achieved vs Age")

heart_disease_var_check

### To visulize the results, we can create a colored prediction map visualizations.
   - Create the grid of variables, and arrange in a data frame
   - Use the fit workflow to make predictions at the grid points
   - bind the predictions as a new column with the grid points
   - plot
       - the colored scatter of the original data
       - the faded colored scatter for the grid points


------------------------------------------------------------------------------------------------------------------


# _4) Expected outcomes and significance:_

- Our classification model aims to provide a reliable and automatic diagnosis for the presence of heart disease. We conduct a comprehensive analysis based on real data collection, a history of common symptoms and medical abnormality detection tests. Multiple predictions then are simultaneously and instantly generated by an automatic and technological-based model. This mechanism reduces potential human errors and time consumption in large data tracing and processing. Therefore, the classification model enhances the reliability, objectivity, and efficiency for heart disease screening.

- While this model aids the detection of present heart disease, the specific subtypes of patients’ heart disease remain undiagnosed. As there are several types of heat-related illness, future questions could advance the prediction and classification of these subtypes. For example, Coronary Artery Disease (CAD) is one of the most common heart diseases. Which further medical tests should be run and which predictors should be used in CAD diagnosis are the possible questions for data analysts and medical researchers to investigate on.
